In [3]:
import pandas as pd
import awswrangler as wr
from sklearn.model_selection import train_test_split
import datetime 
from datetime import date

pd.options.display.max_columns = 999

%matplotlib inline

In [8]:
today = date.today()
day, month, year = (today - datetime.timedelta(days=720)).strftime("%d/%m/%Y").split('/')
period_limit = year + month

today = date.today()
day, month, year = (today - datetime.timedelta(days=60)).strftime("%d/%m/%Y").split('/')
period_potencial = year + month

In [9]:
period_limit, period_potencial

('201905', '202102')

In [5]:
data = wr.s3.read_parquet("s3://rimac-analytics-temporal/individuals/Dante/cross-sell-veh/data/prec/all/data.parquet")


In [12]:
data_potencial = data.loc[(data.periodo_target == int(period_potencial)) & (data.jerarquia == 0)]
data_rest = data.loc[data.periodo_target < int(period_potencial)]

data_drop = data_rest.sort_values(by='jerarquia', ascending=False).drop_duplicates(['periodo_target', 'target', 'cuc']).drop('jerarquia', axis=1)
data_drop = data_drop.sample(frac=1, random_state=0).reset_index(drop=True)
data_train, data_val = train_test_split(data_drop, test_size=0.2, stratify=data_drop.target, random_state=222)

In [13]:
data_train.shape, data_val.shape

((682367, 33), (170592, 33))

In [41]:
data_train.shape, data_val.shape

((602368, 33), (150592, 33))

In [14]:
data_train = data_train.drop(['periodo_target', 'cuc', 'placa'], axis=1, errors='ignore')
data_val = data_val.drop(['periodo_target', 'cuc', 'placa'], axis=1, errors='ignore')

In [16]:
wr.s3.to_parquet(
        df=data_val.reset_index(drop=True),
        path='s3://rimac-analytics-temporal/individuals/Dante/cross-sell-veh/data/prec/valid/data.parquet',
        mode='overwrite',
        dataset=True,
)
wr.s3.to_parquet(
        df=data_train.reset_index(drop=True),
        path='s3://rimac-analytics-temporal/individuals/Dante/cross-sell-veh/data/prec/train/data.parquet',
        mode='overwrite',
        dataset=True,
)
wr.s3.to_parquet(
        df=data_potencial.reset_index(drop=True),
        path='s3://rimac-analytics-temporal/individuals/Dante/cross-sell-veh/data/prec/potencial/data.parquet',
        mode='overwrite',
        dataset=True,
)

{'paths': ['s3://rimac-analytics-temporal/individuals/Dante/cross-sell-veh/data/prec/potencial/data.parquet/c76cfbd3a792412498b6bb8cb6efa7f4.snappy.parquet'],
 'partitions_values': {}}